In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
from scipy import stats
from datetime import timedelta
from collections import Counter
#from warnings import filterwarnings
#filterwarnings('ignore')

In [2]:
data = pd.read_csv('unprofitable_pg4_per_site.dsv', sep=';')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2007 entries, 0 to 2006
Data columns (total 6 columns):
OSP_3_LEAD_P_PG_ID    2007 non-null int64
OSP_4_LEAD_P_PG_ID    2007 non-null int64
SIT_N_KEY_SITE        2007 non-null int64
CM2_REL               2007 non-null object
SALES_BMF             2007 non-null object
ORDERS                2007 non-null int64
dtypes: int64(4), object(2)
memory usage: 94.2+ KB


In [4]:
data['CM2_REL'] = [float(str(i).replace(',', '.')) for i in data['CM2_REL'].values]

In [5]:
data['SALES_BMF'] = [float(str(i).replace(',', '.')) for i in data['SALES_BMF'].values]

In [9]:
data[data['ORDERS'] >= 20].sort_values(by='CM2_REL')

,OSP_3_LEAD_P_PG_ID,OSP_4_LEAD_P_PG_ID,SIT_N_KEY_SITE,CM2_REL,SALES_BMF,ORDERS
1105,251,180361,3,-3243873.39,4829.83,81
528,857,180309,1,-247946.08,6343.23,104
10,2068,2127,12,-241130.27,8840.92,140
519,2068,17963,1,-220348.48,16536.92,273
758,207553,207617,12,-106638.77,15471.62,288
1785,301,10331,1,-17026.97,18861.04,558
247,15,92,4,-11647.09,11053.57,278
1051,305,306,1,-222.19,615.88,20
1788,36,97982,1,-55.24,3778.19,138
27,252,71977,12,-50.54,1196.52,20


In [3]:
import cx_Oracle

In [6]:
query = """select OSP_3_LEAD_P_PG_ID, OSP_4_LEAD_P_PG_ID, sit_n_key_site, round(avg(cm2_perc), 2) cm2_rel, sum(sales) sales_bmf, count(scm_n_order_nr) orders
from (
select scm.scm_n_order_nr, scm.sit_n_key_site, osp.OSP_4_LEAD_P_PG_ID, osp.OSP_3_LEAD_P_PG_ID,
case when sum(SCM_D_ORIG_FINANCE_ORDER_NET) = 0 then 0
else ROUND(sum(SCM_D_FINANCE_CM2) / sum(SCM_D_ORIG_FINANCE_ORDER_NET) * 100, 2)  
end as cm2_perc, 
(sum (scm.SCM_SALES_BMF) - sum(scm.SCM_SALES_CANCEL_BMF)) sales
from ZOOCUBE.DDSTF_SCM_SALES_CONTRIB_MARGIN scm
join ZOOMAN.ORDER_SALES_PROFILE osp
on scm.SCM_N_ORDER_NR = osp.OSP_O_ID
where scm.tdt_t_key_order_date >= sysdate - 15
and(scm.SCM_C_ORDER_STATUS = 'L' or scm.SCM_C_ORDER_STATUS = 'VK')
and osp.osp_ko_count_net = 1
group by scm.SCM_N_ORDER_NR, osp.OSP_3_LEAD_P_PG_ID, osp.OSP_4_LEAD_P_PG_ID, scm.sit_n_key_site
)
group by OSP_3_LEAD_P_PG_ID, OSP_4_LEAD_P_PG_ID, sit_n_key_site
having avg(cm2_perc) <= 10
and sit_n_key_site in (1, 3, 4, 7, 8, 12, 14, 15, 16, 18, 19, 21, 22, 23, 24, 25, 26, 29, 30)
"""

In [7]:
conn = cx_Oracle.connect('zooor', 'zoo12or', 'zoorep', threaded=True, encoding = "UTF-8", nencoding = "UTF-8")
curs = conn.cursor()
pg4 = pd.read_sql(query, con = conn)
conn.close()

In [10]:
pg4.sort_values(by='CM2_REL')

,OSP_3_LEAD_P_PG_ID,OSP_4_LEAD_P_PG_ID,SIT_N_KEY_SITE,CM2_REL,SALES_BMF,ORDERS
516,20,95,26,-10157433.33,0.0002,1
804,251,180361,3,-7962278.44,1790.3072,33
12,2068,2127,12,-553425.14,3865.3197,61
1197,29,323,1,-11901.05,0.4400,1
531,2068,246478,12,-3040.01,1.4500,1
1044,16,2613,12,-2274.51,1.7600,1
1192,18,125586,12,-1359.65,28.4300,2
688,857,17198,12,-1097.89,128.9000,4
539,9,15017,1,-578.47,143.0300,2
1414,274,42317,21,-472.49,5.3964,1


In [42]:
from crontab import CronTab

In [43]:
cron = CronTab(tab="""0 12 * * * command""")
job = cron.new(command='python nc_sp_eligibility_pg4.py')  
cron.write() 

In [44]:
job.is_valid() 

True

In [45]:
for job in cron:
    print(job)

* 12 * * * command
* * * * * python nc_sp_eligibility_pg4.py


In [1]:
import os

In [21]:
title = 'NC_SP_eligibility_PG4_'
date = str(datetime.datetime.now().date())
extension = '.xlsx'

In [23]:
file = title + date + extension
file

'NC_SP_eligibility_PG4_2018-07-27.xlsx'

In [24]:
os.getcwd() + '\\' + file

'C:\\Users\\albertoma\\Desktop\\Savings_plan\\nc_sp_eligibility_on_PG4\\NC_SP_eligibility_PG4_2018-07-27.xlsx'